# Clasificación de Tumores Cerebrales en MRI con Transfer Learning (VGG16)
Este notebook implementa un pipeline de modelado usando transfer learning con VGG16 preentrenada, data augmentation y fine-tuning para clasificación de tumores cerebrales en imágenes MRI.

In [ ]:
# 1. Importar librerías necesarias
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
# 2. Cargar imágenes preprocesadas y preparar splits para transfer learning
DATA_PATH = '../data/preprocesadas'
CLASES = ['brain_glioma', 'brain_menin', 'brain_tumor']
IMG_SIZE = (224, 224)
N_CHANNELS = 3  # Para VGG16
X = []
y = []
for idx, clase in enumerate(CLASES):
    ruta_clase = os.path.join(DATA_PATH, clase)
    imagenes = glob(os.path.join(ruta_clase, '*.jpg'))
    for img_path in imagenes:
        img = Image.open(img_path).convert('L')
        img = img.resize(IMG_SIZE)
        img = np.array(img)
        # Adaptar a 3 canales duplicando
        img_rgb = np.stack([img]*3, axis=-1)
        X.append(img_rgb)
        y.append(idx)
X = np.array(X)
y = np.array(y)
print('Shape X:', X.shape)
print('Shape y:', y.shape)

# Normalizar a [0,1]
X = X.astype('float32') / 255.0

# One-hot encoding de etiquetas
y_cat = to_categorical(y, num_classes=len(CLASES))

# Split train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y_cat, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=np.argmax(y_temp, axis=1), random_state=42)
print(f'Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}')

In [ ]:
# 3. Data augmentation y visualización de ejemplos aumentados
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
 )

# Visualizar algunos ejemplos aumentados
sample_idx = np.random.choice(len(X_train), 1)[0]
img = X_train[sample_idx]
img = np.expand_dims(img, 0)
aug_iter = datagen.flow(img, batch_size=1)
plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    batch = next(aug_iter)[0]
    plt.imshow(batch.astype('float32'))
    plt.axis('off')
plt.suptitle('Ejemplos de data augmentation')
plt.show()

In [ ]:
# 4. Definir modelo VGG16 preentrenado y cabeza personalizada
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in base_model.layers:
    layer.trainable = False  # Congelar capas base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(len(CLASES), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 5. Entrenamiento de la cabeza del modelo (solo capas densas)
batch_size = 32
epochs = 15
callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
    ModelCheckpoint('best_vgg16_head.h5', save_best_only=True, monitor='val_loss')
]

train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
val_gen = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
 )

In [ ]:
# 6. Visualización de resultados y guardado del modelo final
import datetime
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# Curvas de entrenamiento
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.tight_layout()
plt.show()

# Evaluación en test
preds = model.predict(X_test)
y_pred = np.argmax(preds, axis=1)
y_true = np.argmax(y_test, axis=1)

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASES, yticklabels=CLASES)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de confusión en test')
plt.show()

# Reporte de clasificación
print('Reporte de clasificación en test:')
print(classification_report(y_true, y_pred, target_names=CLASES))

In [ ]:
# Guardar el modelo final con precisión de validación en el nombre
MODELOS_DIR = '../models'
os.makedirs(MODELOS_DIR, exist_ok=True)
fecha_hora = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
val_acc_final = max(history.history['val_accuracy'])
nombre_modelo = f"cnn_mri_{fecha_hora}_valacc_{val_acc_final:.4f}.h5"
ruta_modelo = os.path.join(MODELOS_DIR, nombre_modelo)
model.save(ruta_modelo)
print(f"Modelo final guardado en: {ruta_modelo}")